In [52]:
import network_module
from scipy.optimize import minimize
import numpy as np


In [53]:
theta1 = 0.7854              # Alice input 0 (A0)
theta2 = 0          # Bob input 0 (B0 = π/8)
theta3 = -0.3927        # Alice input 1 (A1 = π/4)
theta4 = 0.3927

#random deltas
deltaA = np.random.uniform(-np.pi, np.pi, size=3).tolist()
deltaB = np.random.uniform(-np.pi, np.pi, size=3).tolist()
deltas = [deltaA ,deltaB]

In [54]:
from bayes_opt import BayesianOptimization
import numpy as np

# Simulated CHSH function: peaks near (1.57, 1.57, 1.57) = (π/2, π/2, π/2)
def BayesOPT(deltas,fidelity=1000):
    def simulated_chsh(alpha1, alpha2, alpha3, alpha4, alpha5, alpha6):
    # max_games = 700
    # games = max(800, int(max_games * fidelity))
        g = network_module.Game(deltas = deltas,fid = 0.9)
        alphas_alice = [alpha1,alpha2,alpha3]
        alphas_bob = [alpha4,alpha5,alpha6]
        result = g.start(fidelity,theta1,theta2,theta3,theta4,alphas_alice,alphas_bob)
        return g.referee.compute_chsh_s()

# Define Bayesian optimizer bounds for Alice’s PC angles (voltages mapped to angles)
    pbounds = {
        'alpha1': (-np.pi, np.pi),
        'alpha2': (-np.pi, np.pi),
        'alpha3': (-np.pi, np.pi),
        'alpha4': (-np.pi, np.pi),
        'alpha5': (-np.pi, np.pi),
        'alpha6': (-np.pi, np.pi),
    }

# Create optimizer
    optimizer = BayesianOptimization(
        f=simulated_chsh,
        pbounds=pbounds,
        verbose=1
    )

    import time

    start_time = time.time()
# Run optimization manually with acquisition function
    optimizer.maximize(
        init_points=4,
        n_iter=26
        # acq=utility  # Pass UtilityFunction instance here
    )

# Show best result
    end_time = time.time()
    # print("Total runtime:", end_time - start_time, "seconds")
    # print("Best parameters found:", optimizer.max)
    return list(optimizer.max['params'].values()),end_time - start_time

In [55]:
X_high_fid, time_high_fid = BayesOPT(deltas,fidelity=2500)
alphas_alice = X_high_fid[0:3]
alphas_bob = X_high_fid[3:6]


|   iter    |  target   |  alpha1   |  alpha2   |  alpha3   |  alpha4   |  alpha5   |  alpha6   |
-------------------------------------------------------------------------------------------------
| 2         | 1.2903290 | 1.2409599 | 2.4017721 | 0.5039337 | -2.871016 | -0.444860 | -2.818300 |
| 7         | 2.2643003 | 0.6867709 | 2.1012819 | 0.2203598 | -2.742808 | -0.625656 | -2.559051 |
| 8         | 2.3747551 | 0.8058413 | 1.8848281 | 0.0397769 | -2.538082 | -0.825120 | -2.483050 |


In [56]:
g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
g.start(10000,theta1,theta2,theta3,theta4,alphas_alice,alphas_bob)
S_high = g.referee.compute_chsh_s()
print("S HIGH FID:",S_high)

S HIGH FID: 2.3007197271337407


In [57]:
import numpy as np
from scipy.optimize import minimize

# Number of shots per evaluation
shots = 2500

def objective(x):
    # x = [alpha1, alpha2, alpha3, beta1, beta2, beta3]
    alphas_alice = x[0:3]
    alphas_bob   = x[3:6]

    # Start a new game for each evaluation to avoid stale state
    g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
    g.start(shots, theta1, theta2, theta3, theta4, alphas_alice, alphas_bob)

    S = g.referee.compute_chsh_s()

    return -S  # minimize negative S to maximize S

# Construct COBYLA inequality constraints for [-pi, pi]
pi_val = np.pi
cons = []
for i in range(6):
    # x[i] >= -pi  ->  x[i] + pi >= 0
    cons.append({'type': 'ineq', 'fun': lambda x, i=i: x[i] + pi_val})
    # x[i] <= pi   ->  pi - x[i] >= 0
    cons.append({'type': 'ineq', 'fun': lambda x, i=i: pi_val - x[i]})

# Initial guess (can be 0s or random within [-pi, pi])
x0 = np.zeros(6)

res = minimize(objective, x0, method='COBYLA',
               constraints=cons,
               options={'rhobeg': 1.0, 'tol': 1e-6, 'maxiter': 30})

print("Best angles (Alice+Bob):", res.x)
print("Max S:", -res.fun)
print("Success:", res.success)



Best angles (Alice+Bob): [ 0.9808153   1.07158074  1.56365513 -0.41509132  0.04921739  0.05482492]
Max S: 2.476154571905793
Success: False


In [58]:
alphas_alice = res.x[0:3]
alphas_bob = res.x[3:]
g = network_module.Game(deltas = deltas,guarantee_entanglement = True,fid = 0.9)
g.start(10000, theta1, theta2, theta3, theta4, alphas_alice, alphas_bob)
S = g.referee.compute_chsh_s()

In [59]:
S

2.4226814308211035